In [1]:
import os
import hashlib
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import utils as asym_utils


In [3]:
DATASET_DIR = 'dataset'
OUTPUT_DIR = 'encrypted_dataset'

# Geração de chaves RSA
def generate_rsa_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()

    # Salva as chaves em disco
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))

    with open("public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))
    return private_key, public_key


def encrypt_image(img_path, public_key, private_key):
    with open(img_path, "rb") as file:
        image_data = file.read()

    fernet_key = Fernet.generate_key()
    fernet = Fernet(fernet_key)

    encrypted_image = fernet.encrypt(image_data)

    encrypted_key = public_key.encrypt(
        fernet_key,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )

    # 4. Assinar hash da imagem com chave privada
    digest = hashlib.sha256(image_data).digest()
    signature = private_key.sign(
        digest,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )

    return encrypted_image, encrypted_key, signature


# Processa todas as imagens do dataset
def encrypt_dataset():
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    private_key, public_key = generate_rsa_keys()

    for split in ['train', 'val']:
        for label in ['deficiente', 'normal']:
            input_dir = os.path.join(DATASET_DIR, split, label)
            output_subdir = os.path.join(OUTPUT_DIR, split, label)
            os.makedirs(output_subdir, exist_ok=True)

            for img_name in os.listdir(input_dir):
                img_path = os.path.join(input_dir, img_name)
                enc_img, enc_key, signature = encrypt_image(img_path, public_key, private_key)

                base = os.path.join(output_subdir, img_name)

                with open(base + '.enc', 'wb') as f:
                    f.write(enc_img)

                with open(base + '.key', 'wb') as f:
                    f.write(enc_key)

                with open(base + '.sig', 'wb') as f:
                    f.write(signature)

encrypt_dataset()


In [4]:

ENCRYPTED_DIR = 'encrypted_dataset'
DECRYPTED_DIR = 'decrypted_dataset'

# Carrega chave privada RSA
def load_private_key(path="private_key.pem"):
    with open(path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None
        )
    return private_key

def load_public_key(path="public_key.pem"):
    with open(path, "rb") as key_file:
        public_key = serialization.load_pem_public_key(key_file.read())
    return public_key


# Desencripta uma imagem com chave Fernet recuperada com RSA
def decrypt_image(encrypted_img_path, encrypted_key_path, sig_path, private_key, public_key):
    # 1. Carregar chave Fernet criptografada
    with open(encrypted_key_path, 'rb') as f:
        encrypted_key = f.read()

    # 2. Descriptografar chave Fernet com RSA
    fernet_key = private_key.decrypt(
        encrypted_key,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )

    fernet = Fernet(fernet_key)

    # 3. Descriptografar imagem
    with open(encrypted_img_path, 'rb') as f:
        encrypted_img = f.read()
    decrypted_img = fernet.decrypt(encrypted_img)

    # 4. Verificar assinatura digital
    with open(sig_path, 'rb') as f:
        signature = f.read()

    digest = hashlib.sha256(decrypted_img).digest()

    try:
        public_key.verify(
            signature,
            digest,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        print(f"[OK] Assinatura válida: {os.path.basename(encrypted_img_path)}")
    except Exception as e:
        print(f"[ERRO] Assinatura inválida: {os.path.basename(encrypted_img_path)}")

    return decrypted_img


# Percorre diretório e descriptografa
def decrypt_dataset():
    private_key = load_private_key()
    public_key = load_public_key()  # Nova função

    for split in ['train', 'val']:
        for label in ['deficiente', 'normal']:
            input_dir = os.path.join(ENCRYPTED_DIR, split, label)
            output_dir = os.path.join(DECRYPTED_DIR, split, label)
            os.makedirs(output_dir, exist_ok=True)

            for file in os.listdir(input_dir):
                if file.endswith('.enc'):
                    name = file[:-4]
                    img_path = os.path.join(input_dir, name + '.enc')
                    key_path = os.path.join(input_dir, name + '.key')
                    sig_path = os.path.join(input_dir, name + '.sig')

                    decrypted_img = decrypt_image(img_path, key_path, sig_path, private_key, public_key)

                    with open(os.path.join(output_dir, name), 'wb') as f:
                        f.write(decrypted_img)


decrypt_dataset()


[OK] Assinatura válida: Fe (1).JPG.enc
[OK] Assinatura válida: Fe (10).jpg.enc
[OK] Assinatura válida: Fe (11).jpg.enc
[OK] Assinatura válida: Fe (12).jpg.enc
[OK] Assinatura válida: Fe (13).jpg.enc
[OK] Assinatura válida: Fe (14).jpg.enc
[OK] Assinatura válida: Fe (15).jpg.enc
[OK] Assinatura válida: Fe (16).jpg.enc
[OK] Assinatura válida: Fe (17).jpg.enc
[OK] Assinatura válida: Fe (18).jpg.enc
[OK] Assinatura válida: Fe (19).jpg.enc
[OK] Assinatura válida: Fe (2).JPG.enc
[OK] Assinatura válida: Fe (20).jpg.enc
[OK] Assinatura válida: Fe (21).jpg.enc
[OK] Assinatura válida: Fe (22).jpg.enc
[OK] Assinatura válida: Fe (23).jpg.enc
[OK] Assinatura válida: Fe (24).jpg.enc
[OK] Assinatura válida: Fe (25).jpg.enc
[OK] Assinatura válida: Fe (26).jpg.enc
[OK] Assinatura válida: Fe (27).jpg.enc
[OK] Assinatura válida: Fe (28).jpg.enc
[OK] Assinatura válida: Fe (29).jpg.enc
[OK] Assinatura válida: Fe (3).JPG.enc
[OK] Assinatura válida: Fe (30).jpg.enc
[OK] Assinatura válida: Fe (31).jpg.enc
[OK